In [207]:
ENV["PYTHONPATH"] = "/home/gridsan/aligho/.local/lib/python3.8/site-packages/PyNormaliz-2.15-py3.8-linux-x86_64.egg";

In [208]:
using Crystalline, MPBUtils, JLD2, LinearAlgebra, StaticArrays, SymmetryBases, PyPlot, DelimitedFiles, PrettyTables
using Crystalline: symvec2string
include("./symeigs/corner_charges.jl");
include("./symeigs/wyckoffs_dict.jl");

In [213]:
# spacegroup 2
sg = 2
id = 1
polarization = "tm"
brs = bandreps(sg, 2)
band_groupingsv = Vector{BandSummary}[]
for id in 1:4
    symeigsd, lgd = read_symdata("dim2-sg$sg-$id-res64-$polarization", dir="./symeigs/shiftedlattices/output");
    lgirsd = pick_lgirreps(lgd)
    fixup_gamma_symmetry!(symeigsd, lgd, Symbol(uppercase(polarization)))
    band_groupings = analyze_symmetry_data(symeigsd, lgirsd, brs);
    push!(band_groupingsv, band_groupings)
end

In [214]:
# We now go through each band grouping one-by-one and make sure that the irreps change as we expect they would
for (i, j, k, l) in zip(band_groupingsv...)
    base_n = i.n
    shift_0_1 = base_n[[2, 1, 3, 4, 6, 5, 7, 8, 9]] # Y and A irreps change but not the B irreps
    shift_1_0 = base_n[[1, 2, 4, 3, 6, 5, 7, 8, 9]] # B and A irreps change but not the Y irreps
    shift_1_1 = base_n[[2, 1, 4, 3, 5, 6, 7, 8, 9]] # Y and B irreps change but not the A irreps

    @assert j.n == shift_0_1
    @assert k.n == shift_1_0
    @assert l.n == shift_1_1
end

In [215]:
ws = wyckoffs(sg, 2)
for (i, j, k, l) in zip(band_groupingsv...)
    @assert corners(i, ws[4]) == corners(j)
    @assert corners(i, ws[3]) == corners(k)
    @assert corners(i, ws[2]) == corners(l)
    @assert polarizations(i, ws[4]) == polarizations(j)
    @assert polarizations(i, ws[3]) == polarizations(k)
    @assert polarizations(i, ws[2]) == polarizations(l)
end

In [216]:
# spacegroup 6
sg = 6
id = 1
polarization = "tm"
brs = bandreps(sg, 2)
band_groupingsv = Vector{BandSummary}[]
for id in 1:4
    symeigsd, lgd = read_symdata("dim2-sg$sg-$id-res64-$polarization", dir="./symeigs/shiftedlattices/output");
    lgirsd = pick_lgirreps(lgd)
    fixup_gamma_symmetry!(symeigsd, lgd, Symbol(uppercase(polarization)))
    band_groupings = analyze_symmetry_data(symeigsd, lgirsd, brs);
    push!(band_groupingsv, band_groupings)
end

In [217]:
for (i, j, k, l) in zip(band_groupingsv...)
    base_n = i.n
    shift_0_1 = base_n[[4, 3, 2, 1, 8, 7, 6, 5, 9, 10, 11, 12, 13, 14, 15, 16, 17]]
    shift_1_0 = base_n[[1, 2, 3, 4, 7, 8, 5, 6, 11, 12, 9, 10, 13, 14, 15, 16, 17]]
    shift_1_1 = base_n[[4, 3, 2, 1, 6, 5, 8, 7, 11, 12, 9, 10, 13, 14, 15, 16, 17]]

    @assert j.n == shift_0_1
    @assert k.n == shift_1_0
    @assert l.n == shift_1_1
end

In [8]:
ws = wyckoffs(sg, 2)
for (i, j, k, l) in zip(band_groupingsv...)
    @assert corners(i, ws[8]) == corners(j)
    @assert corners(i, ws[7]) == corners(k)
    @assert corners(i, ws[6]) == corners(l)
    @assert polarizations(i, ws[8]) == polarizations(j)
    @assert polarizations(i, ws[7]) == polarizations(k)
    @assert polarizations(i, ws[6]) == polarizations(l)
end

In [9]:
# spacegroup 9
sg = 9
id = 1
polarization = "tm"
brs = bandreps(sg, 2)
band_groupingsv9 = Vector{BandSummary}[]
for id in 1:2
    symeigsd, lgd = read_symdata("dim2-sg$sg-$id-res64-$polarization", sgnum=9, dir="./symeigs/shiftedlattices/output");
    lgirsd = pick_lgirreps(lgd)
    fixup_gamma_symmetry!(symeigsd, lgd, Symbol(uppercase(polarization)))
    band_groupings = analyze_symmetry_data(symeigsd, lgirsd, brs);
    push!(band_groupingsv9, band_groupings)
end

In [10]:
for (i, j) in zip(band_groupingsv9...)
    base_n = i.n
    shift = base_n[[1, 2, 3, 4, 6, 5, 7, 8, 9, 10, 11]]
    @assert j.n == shift
end

In [11]:
ws = wyckoffs(sg, 2)
for (i, j) in zip(band_groupingsv9...)
    @assert corners(i, ws[5]) == corners(j)
    @assert polarizations(i, ws[5]) == polarizations(j)
end

In [12]:
sg = 9
id = 1
polarization = "tm"
brs = bandreps(2, 2)
band_groupingsv2 = Vector{BandSummary}[]
for id in 3:5
    symeigsd, lgd = read_symdata("dim2-sg$sg-$id-res64-$polarization", sgnum=2, dir="./symeigs/shiftedlattices/output");
    lgirsd = pick_lgirreps(lgd)
    fixup_gamma_symmetry!(symeigsd, lgd, Symbol(uppercase(polarization)))
    band_groupings = analyze_symmetry_data(symeigsd, lgirsd, brs);
    push!(band_groupingsv2, band_groupings)
end

In [13]:
for grouping in band_groupingsv9[1]
    band = grouping.band
    n9 = grouping.n
    n2 = sum(band_groupingsv2[3][band]).n
    @assert n2 == [n9[5], n9[6], n9[5], n9[6], n9[1]+n9[2], n9[3]+n9[4], n9[7]+n9[8], n9[9]+n9[10], n9[11]]
end

for grouping in band_groupingsv9[1]
    band = grouping.band
    n9 = grouping.n
    n2 = sum(band_groupingsv2[1][band]).n
    @assert n2 == [n9[6], n9[5], n9[5], n9[6], n9[3]+n9[4], n9[1]+n9[2], n9[7]+n9[8], n9[9]+n9[10], n9[11]]
end

for grouping in band_groupingsv9[1]
    band = grouping.band
    n9 = grouping.n
    n2 = sum(band_groupingsv2[2][band]).n
    @assert n2 == [n9[5], n9[6], n9[6], n9[5], n9[3]+n9[4], n9[1]+n9[2], n9[7]+n9[8], n9[9]+n9[10], n9[11]]
end

In [14]:
for grouping in band_groupingsv9[1]
    band = grouping.band
    grouping_c2 = sum(band_groupingsv2[3][band])
    @assert corners(grouping) == corners(grouping_c2) 
    @assert polarizations(grouping) == polarizations(grouping_c2)    
end
for grouping in band_groupingsv9[1]
    wp = sg_wyckoffs_dict[9][3]
    band = grouping.band
    grouping_c2 = sum(band_groupingsv2[1][band])
    @assert corners(grouping, wp) == corners(grouping_c2) 
    @assert polarizations(grouping, wp) == polarizations(grouping_c2)    
end
for grouping in band_groupingsv9[1]
    wp = sg_wyckoffs_dict[9][4]
    band = grouping.band
    grouping_c2 = sum(band_groupingsv2[2][band])
    @assert corners(grouping, wp) == corners(grouping_c2) 
    @assert polarizations(grouping, wp) == polarizations(grouping_c2)    
end

In [15]:
# spacegroup 10
sg = 10
id = 1
polarization = "tm"
brs = bandreps(sg, 2)
band_groupingsv = Vector{BandSummary}[]
for id in 1:2
    symeigsd, lgd = read_symdata("dim2-sg$sg-$id-res64-$polarization", dir="./symeigs/shiftedlattices/output");
    lgirsd = pick_lgirreps(lgd)
    fixup_gamma_symmetry!(symeigsd, lgd, Symbol(uppercase(polarization)))
    band_groupings = analyze_symmetry_data(symeigsd, lgirsd, brs);
    push!(band_groupingsv, band_groupings)
end

In [16]:
for (i, j) in zip(band_groupingsv...)
    base_n = i.n
    shift = base_n[[2, 1, 4, 3, 5, 6, 7, 8, 9]]
    @assert j.n == shift
end

In [17]:
ws = wyckoffs(sg, 2)
for (i, j) in zip(band_groupingsv...)
    @assert corners(i, ws[3]) == corners(j)
    @assert polarizations(i, ws[3]) == polarizations(j)
end

In [18]:
# spacegroup 11
sg = 11
id = 1
polarization = "te"
brs = bandreps(sg, 2)
band_groupingsv = Vector{BandSummary}[]
for id in 1:2
    symeigsd, lgd = read_symdata("dim2-sg$sg-$id-res64-$polarization", dir="./symeigs/shiftedlattices/output");
    lgirsd = pick_lgirreps(lgd)
    fixup_gamma_symmetry!(symeigsd, lgd, Symbol(uppercase(polarization)))
    band_groupings = analyze_symmetry_data(symeigsd, lgirsd, brs);
    push!(band_groupingsv, band_groupings)
end

In [19]:
for (i, j) in zip(band_groupingsv...)
    base_n = i.n
    shift = base_n[[4, 3, 2, 1, 7, 8, 5, 6, 9, 10, 11, 12, 13, 14, 15]]
    @assert j.n == shift
end

In [20]:
ws = wyckoffs(sg, 2)
for (i, j) in zip(band_groupingsv...)
    @assert corners(i, ws[6]) == corners(j)
    @assert polarizations(i, ws[6]) == polarizations(j)
end

In [21]:
# spacegroup 12
sg = 12
id = 1
polarization = "tm"
brs = bandreps(sg, 2)
band_groupingsv = Vector{BandSummary}[]
for id in 1:2
    symeigsd, lgd = read_symdata("dim2-sg$sg-$id-res64-$polarization", dir="./symeigs/shiftedlattices/output");
    lgirsd = pick_lgirreps(lgd)
    fixup_gamma_symmetry!(symeigsd, lgd, Symbol(uppercase(polarization)))
    band_groupings = analyze_symmetry_data(symeigsd, lgirsd, brs);
    push!(band_groupingsv, band_groupings)
end

In [22]:
for (i, j) in zip(band_groupingsv...)
    base_n = i.n
    shift = base_n
    @assert j.n == shift
end

In [218]:
# spacegroup 13
sg = 13
id = 1
polarization = "tm"
brs = bandreps(sg, 2)
band_groupingsv = Vector{BandSummary}[]
band_groupingsv_flipped_k = Vector{BandSummary}[]
for id in 1:3
    filename = "dim2-sg$sg-$id-res64-$polarization" 
    symeigsd, lgd = read_symdata(filename, dir="./symeigs/shiftedlattices/output", flip_ksign=false);
    lgirsd = pick_lgirreps(lgd)
    fixup_gamma_symmetry!(symeigsd, lgd, Symbol(uppercase(polarization)))
    band_groupings = analyze_symmetry_data(symeigsd, lgirsd, brs);
    push!(band_groupingsv, band_groupings)

    filename = "dim2-sg$sg-$id-res64-$polarization-flipk"
    symeigsd, lgd = read_symdata(filename, dir="./symeigs/shiftedlattices/output", flip_ksign=true);
    lgirsd = pick_lgirreps(lgd)
    fixup_gamma_symmetry!(symeigsd, lgd, Symbol(uppercase(polarization)))
    band_groupings = analyze_symmetry_data(symeigsd, lgirsd, brs);
    push!(band_groupingsv_flipped_k, band_groupings)
end

In [111]:
brs_labs = ["(" * br.wyckpos * "|"  * prod(string.(collect(br.label)[1:end-2])) * ")" for br in brs]
for (summary_1a, summary_1b, summary_1c, summary_1a_flipped_k, summary_1b_flipped_k, summary_1c_flipped_k) in
    zip(band_groupingsv..., band_groupingsv_flipped_k...)
    
    brs_decomp_1a = decompose(summary_1a.n, brs)
    brs_decomp_1a_flipped_k = decompose(summary_1a_flipped_k.n, brs)

    decomp_1a = Crystalline.symvec2string(brs_decomp_1a, brs_labs)
    decomp_1a_flipped_k = Crystalline.symvec2string(brs_decomp_1a_flipped_k, brs_labs)

    brs_decomp_1b = decompose(summary_1b.n, brs)
    brs_decomp_1b_flipped_k = decompose(summary_1b_flipped_k.n, brs)

    decomp_1b = Crystalline.symvec2string(brs_decomp_1b, brs_labs)
    decomp_1b_flipped_k = Crystalline.symvec2string(brs_decomp_1b_flipped_k, brs_labs)

    brs_decomp_1c = decompose(summary_1c.n, brs)
    brs_decomp_1c_flipped_k = decompose(summary_1c_flipped_k.n, brs)

    decomp_1c = Crystalline.symvec2string(brs_decomp_1c, brs_labs)
    decomp_1c_flipped_k = Crystalline.symvec2string(brs_decomp_1c_flipped_k, brs_labs)

    println("Decomposition at 1a: ", decomp_1a, " flipped_k : ", decomp_1a_flipped_k)
    println("Decomposition at 1b: ", decomp_1b, " flipped_k : ", decomp_1b_flipped_k)
    println("Decomposition at 1c: ", decomp_1c, " flipped_k : ", decomp_1c_flipped_k, "\n")
end

Decomposition at 1a: [(1b|A)] flipped_k : [(1c|A)]
Decomposition at 1b: [(1c|A)] flipped_k : [(1b|A)]
Decomposition at 1c: [(1a|A)] flipped_k : [(1a|A)]

Decomposition at 1a: [(1c|A)] flipped_k : [(1b|A)]
Decomposition at 1b: [(1a|A)] flipped_k : [(1a|A)]
Decomposition at 1c: [(1b|A)] flipped_k : [(1c|A)]

Decomposition at 1a: [(1b|E)] flipped_k : [(1c|E)]
Decomposition at 1b: [(1c|E)] flipped_k : [(1b|E)]
Decomposition at 1c: [(1a|E)] flipped_k : [(1a|E)]

Decomposition at 1a: [(1b|A)] flipped_k : [(1c|A)]
Decomposition at 1b: [(1c|A)] flipped_k : [(1b|A)]
Decomposition at 1c: [(1a|A)] flipped_k : [(1a|A)]

Decomposition at 1a: [(1c|E)-(1b|A)+(1a|A)] flipped_k : [(1c|E)-(1b|A)+(1a|A)]
Decomposition at 1b: [(1c|E)+(1b|A)-(1a|A)] flipped_k : [2.0(1c|A)+(1c|E)-(1b|A)-(1a|A)]
Decomposition at 1c: [2.0(1c|A)+(1c|E)-(1b|A)-(1a|A)] flipped_k : [(1c|E)+(1b|A)-(1a|A)]

Decomposition at 1a: [(1c|A)] flipped_k : [(1b|A)]
Decomposition at 1b: [(1a|A)] flipped_k : [(1a|A)]
Decomposition at 1c: [(1

In [219]:
ws = wyckoffs(sg, 2)
for (i, j, k) in zip(band_groupingsv...)
    @assert corners(i, ws[3]) == corners(j)
    @assert corners(i, ws[2]) == corners(k)
    @assert polarizations(i, ws[3]) == polarizations(j)
    @assert polarizations(i, ws[2]) == polarizations(k)
end

In [233]:
# spacegroup 14
sg = 14
id = 1
polarization = "tm"
brs = bandreps(sg, 2)
band_groupingsv = Vector{BandSummary}[]
band_groupingsv_flipped_k = Vector{BandSummary}[]
for id in 1:3
    filename = "dim2-sg$sg-$id-res64-$polarization"
    symeigsd, lgd = read_symdata(filename, dir="./symeigs/shiftedlattices/output", flip_ksign = false);
    lgirsd = pick_lgirreps(lgd)
    fixup_gamma_symmetry!(symeigsd, lgd, Symbol(uppercase(polarization)))
    band_groupings = analyze_symmetry_data(symeigsd, lgirsd, brs);
    push!(band_groupingsv, band_groupings)
    
    filename = "dim2-sg$sg-$id-res64-$polarization-flipk"
    symeigsd, lgd = read_symdata(filename, dir="./symeigs/shiftedlattices/output", flip_ksign = true);
    lgirsd = pick_lgirreps(lgd)
    fixup_gamma_symmetry!(symeigsd, lgd, Symbol(uppercase(polarization)))
    band_groupings = analyze_symmetry_data(symeigsd, lgirsd, brs);
    push!(band_groupingsv_flipped_k, band_groupings)
end

In [234]:
brs_labs = ["(" * br.wyckpos * "|"  * prod(string.(collect(br.label)[1:end-2])) * ")" for br in brs]
for (summary_1a, summary_1b, summary_1c, summary_1a_flipped_k, summary_1b_flipped_k, summary_1c_flipped_k) in
    zip(band_groupingsv..., band_groupingsv_flipped_k...)
    
    brs_decomp_1a = decompose(summary_1a.n, brs)
    brs_decomp_1a_flipped_k = decompose(summary_1a_flipped_k.n, brs)

    decomp_1a = Crystalline.symvec2string(brs_decomp_1a, brs_labs)
    decomp_1a_flipped_k = Crystalline.symvec2string(brs_decomp_1a_flipped_k, brs_labs)

    brs_decomp_1b = decompose(summary_1b.n, brs)
    brs_decomp_1b_flipped_k = decompose(summary_1b_flipped_k.n, brs)

    decomp_1b = Crystalline.symvec2string(brs_decomp_1b, brs_labs)
    decomp_1b_flipped_k = Crystalline.symvec2string(brs_decomp_1b_flipped_k, brs_labs)

    brs_decomp_1c = decompose(summary_1c.n, brs)
    brs_decomp_1c_flipped_k = decompose(summary_1c_flipped_k.n, brs)

    decomp_1c = Crystalline.symvec2string(brs_decomp_1c, brs_labs)
    decomp_1c_flipped_k = Crystalline.symvec2string(brs_decomp_1c_flipped_k, brs_labs)

    println("Decomposition at 1a: ", decomp_1a, " flipped_k : ", decomp_1a_flipped_k)
    println("Decomposition at 1b: ", decomp_1b, " flipped_k : ", decomp_1b_flipped_k)
    println("Decomposition at 1c: ", decomp_1c, " flipped_k : ", decomp_1c_flipped_k, "\n")
end

Decomposition at 1a: [(1a|A₁)] flipped_k : [(1a|A₁)]
Decomposition at 1b: [(1b|A₁)] flipped_k : [(1c|A₁)]
Decomposition at 1c: [(1c|A₁)] flipped_k : [(1b|A₁)]

Decomposition at 1a: [(1a|E)] flipped_k : [(1a|E)]
Decomposition at 1b: [(1b|E)] flipped_k : [(1c|E)]
Decomposition at 1c: [(1c|E)] flipped_k : [(1b|E)]

Decomposition at 1a: [(1b|A₁)] flipped_k : [(1c|A₁)]
Decomposition at 1b: [(1c|A₁)] flipped_k : [(1b|A₁)]
Decomposition at 1c: [(1a|A₁)] flipped_k : [(1a|A₁)]

Decomposition at 1a: [(1b|E)] flipped_k : [(1c|E)]
Decomposition at 1b: [(1c|E)] flipped_k : [(1b|E)]
Decomposition at 1c: [(1a|E)] flipped_k : [(1a|E)]

Decomposition at 1a: [(1c|A₁)] flipped_k : [(1b|A₁)]
Decomposition at 1b: [(1a|A₁)] flipped_k : [(1a|A₁)]
Decomposition at 1c: [(1b|A₁)] flipped_k : [(1c|A₁)]

Decomposition at 1a: [(1b|A₂)] flipped_k : [(1c|A₂)]
Decomposition at 1b: [(1c|A₂)] flipped_k : [(1b|A₂)]
Decomposition at 1c: [(1a|A₂)] flipped_k : [(1a|A₂)]

Decomposition at 1a: [(1b|E)+(1a|A₁)] flipped_k : [(

In [244]:
# spacegroup 14
sg = 14
id = 1
polarization = "tm"
brs = bandreps(13, 2)
band_groupingsv13 = Vector{BandSummary}[]
band_groupingsv13_flipped_k = Vector{BandSummary}[]

symeigsdv = []
for id in 4:4
    symeigsd, lgd = read_symdata("dim2-sg$sg-$id-res64-$polarization", sgnum = 13, dir="./symeigs/shiftedlattices/output");
    lgirsd = pick_lgirreps(lgd)
    fixup_gamma_symmetry!(symeigsd, lgd, Symbol(uppercase(polarization)))
    push!(symeigsdv, symeigsd)
    band_groupings = analyze_symmetry_data(symeigsd, lgirsd, brs);
    push!(band_groupingsv13, band_groupings)

    symeigsd, lgd = read_symdata("dim2-sg$sg-$id-res64-$polarization-flipk", sgnum = 13, dir="./symeigs/shiftedlattices/output", flip_ksign=true);
    lgirsd = pick_lgirreps(lgd)
    fixup_gamma_symmetry!(symeigsd, lgd, Symbol(uppercase(polarization)))
    push!(symeigsdv, symeigsd)
    band_groupings = analyze_symmetry_data(symeigsd, lgirsd, brs);
    push!(band_groupingsv13_flipped_k, band_groupings)
end

In [223]:
for (i, j, k) in zip(band_groupingsv...)
    base_n = i.n
    shift_1 = base_n[[1, 2, 3, 4, 5, 8, 6, 7, 9]]
    shift_2 = base_n[[1, 2, 3, 4, 5, 7, 8, 6, 9]]

    @assert j.n == shift_1
    @assert k.n == shift_2
end

In [225]:
ws = wyckoffs(sg, 2)
for (i, j, k) in zip(band_groupingsv...)
    @assert corners(i, ws[4]) == corners(j)
    @assert corners(i, ws[3]) == corners(k)
    @assert polarizations(i, ws[4]) == polarizations(j)
    @assert polarizations(i, ws[3]) == polarizations(k)
end

In [247]:
#Spacegroup 15

#In spacegroup 15, we have several Wyckoff positions as candidate centers.
#However, only the (0, 0) position respects all #spacegroup 15 symmetries, while the others respect only the C3 symmetry.
#Therefore, we analyze spacegroup 15 in the #subgroup, c3. 

sg = 15
polarization = "tm"
brs_13 = bandreps(13, 2)
brs_15 = bandreps(15, 2)
band_groupingsv_13 = Vector{BandSummary}[]
band_groupingsv_15 = Vector{BandSummary}[]

band_groupingsv_13_flipped_k = Vector{BandSummary}[]
band_groupingsv_15_flipped_k = Vector{BandSummary}[]


for id in 2:4
    symeigsd, lgd = read_symdata("dim2-sg$sg-$id-res64-$polarization", sgnum=13, dir="./symeigs/shiftedlattices/output");
    lgirsd = pick_lgirreps(lgd)
    fixup_gamma_symmetry!(symeigsd, lgd, Symbol(uppercase(polarization)))
    band_groupings = analyze_symmetry_data(symeigsd, lgirsd, brs_13);
    push!(band_groupingsv_13, band_groupings)

    symeigsd, lgd = read_symdata("dim2-sg$sg-$id-res64-$polarization-flipk", sgnum=13, dir="./symeigs/shiftedlattices/output", flip_ksign=true);
    lgirsd = pick_lgirreps(lgd)
    fixup_gamma_symmetry!(symeigsd, lgd, Symbol(uppercase(polarization)))
    band_groupings = analyze_symmetry_data(symeigsd, lgirsd, brs_13);
    push!(band_groupingsv_13_flipped_k, band_groupings)
end

for id in 1:1
    symeigsd, lgd = read_symdata("dim2-sg$sg-$id-res64-$polarization", dir="./symeigs/shiftedlattices/output");
    lgirsd = pick_lgirreps(lgd)
    fixup_gamma_symmetry!(symeigsd, lgd, Symbol(uppercase(polarization)))
    band_groupings = analyze_symmetry_data(symeigsd, lgirsd, brs_15);
    push!(band_groupingsv_15, band_groupings)

    symeigsd, lgd = read_symdata("dim2-sg$sg-$id-res64-$polarization-flipk", dir="./symeigs/shiftedlattices/output", flip_ksign=true);
    lgirsd = pick_lgirreps(lgd)
    fixup_gamma_symmetry!(symeigsd, lgd, Symbol(uppercase(polarization)))
    band_groupings = analyze_symmetry_data(symeigsd, lgirsd, brs_15);
    push!(band_groupingsv_15_flipped_k, band_groupings)
end

In [254]:
brs_labs_15 = ["(" * br.wyckpos * "|"  * prod(string.(collect(br.label)[1:end-2])) * ")" for br in brs_15]
brs_labs_13 = ["(" * br.wyckpos * "|"  * prod(string.(collect(br.label)[1:end-2])) * ")" for br in brs_13]

for (summary_1a, summary_1b, summary_1c, summary_1a_flipped_k, summary_1b_flipped_k, summary_1c_flipped_k) in
    zip(band_groupingsv_15..., band_groupingsv_13[1:2]..., band_groupingsv_15_flipped_k..., band_groupingsv_13_flipped_k[1:2]...)
    (summary_1a.n[end] !=  summary_1b.n[end]) && break

    brs_decomp_1a = decompose(summary_1a.n, brs_15)
    brs_decomp_1a_flipped_k = decompose(summary_1a_flipped_k.n, brs_15)

    decomp_1a = Crystalline.symvec2string(brs_decomp_1a, brs_labs_15)
    decomp_1a_flipped_k = Crystalline.symvec2string(brs_decomp_1a_flipped_k, brs_labs_15)

    brs_decomp_1b = decompose(summary_1b.n, brs_13)
    brs_decomp_1b_flipped_k = decompose(summary_1b_flipped_k.n, brs_13)

    decomp_1b = Crystalline.symvec2string(brs_decomp_1b, brs_labs_13)
    decomp_1b_flipped_k = Crystalline.symvec2string(brs_decomp_1b_flipped_k, brs_labs_13)

    brs_decomp_1c = decompose(summary_1c.n, brs_13)
    brs_decomp_1c_flipped_k = decompose(summary_1c_flipped_k.n, brs_13)

    decomp_1c = Crystalline.symvec2string(brs_decomp_1c, brs_labs_13)
    decomp_1c_flipped_k = Crystalline.symvec2string(brs_decomp_1c_flipped_k, brs_labs_13)

    println("Decomposition at 1a: ", decomp_1a, " flipped_k: ", decomp_1a_flipped_k)
    println("Decomposition at 1b: ", decomp_1b, " flipped_k: ", decomp_1b_flipped_k)
    println("Decomposition at 1c: ", decomp_1c, " flipped_k: ", decomp_1c_flipped_k, "\n")
end

Decomposition at 1a: [(1a|A₁)] flipped_k: [(1a|A₁)]
Decomposition at 1b: [(1b|A)] flipped_k: [(1c|A)]
Decomposition at 1c: [(1c|A)] flipped_k: [(1b|A)]

Decomposition at 1a: [(1a|E)] flipped_k: [(1a|E)]
Decomposition at 1b: [(1b|E)] flipped_k: [(1c|E)]
Decomposition at 1c: [(1c|E)] flipped_k: [(1b|E)]



In [33]:
for (idx, (j, k, i)) in enumerate(zip(band_groupingsv_13...))
    idx > 10 && break
    base_n = i.n
    shift_1 = base_n[[2, 3, 1, 4, 5, 8, 6, 7, 9]]
    shift_2 = base_n[[3, 1, 2, 4, 5, 7, 8, 6, 9]]

    @assert j.n == shift_1
    @assert k.n == shift_2
end

In [34]:
ws = wyckoffs(13, 2)
for (i, j, k, l) in zip(cumsum.(band_groupingsv_13)..., cumsum.(band_groupingsv_15)...)
    k.n[end] == l.n[end] || continue
    @assert corners(k) == corners(l, ws[4])
    @assert polarizations(k, ws[4]) == polarizations(l)
end
for (i, j, k, l) in zip(cumsum.(band_groupingsv_13)..., cumsum.(band_groupingsv_15)...)
    k.n[end] == l.n[end] || continue
    @assert corners(i) == corners(l, ws[3])
    @assert polarizations(i) == polarizations(l, ws[3])
end
for (i, j, k, l) in zip(cumsum.(band_groupingsv_13)..., cumsum.(band_groupingsv_15)...)
    j.n[end] == l.n[end] || continue
    @assert corners(j) == corners(l, ws[2])
    @assert polarizations(j) == polarizations(l, ws[2])
end

In [90]:
brs_15 = bandreps(15, 2)
brs_13 = bandreps(13, 2);

In [91]:
# Below, we check that the band representations in spacegroup 15 have the corner charges we'd expect

In [35]:
for wyckoff in wyckoffs(13, 2)[end-2:end]
    a  = corner_sg13(brs_13[1].irvec, brs_13[1].irlabs, wyckoff)
    b  = corner_sg13(brs_13[3].irvec, brs_13[1].irlabs, wyckoff)
    @assert corner_sg15(brs_15[1].irvec, brs_15[1].irlabs, wyckoff) == a + b
end

In [36]:
for wyckoff in wyckoffs(13, 2)[end-2:end]
    @assert corner_sg15(brs_15[2].irvec, brs_15.irlabs, wyckoff) == corner_sg13(brs_13[2].irvec + brs_13[4].irvec, brs_13.irlabs, wyckoff)
end

In [37]:
for wyckoff in wyckoffs(13, 2)[end-2:end]
    @assert corner_sg15(brs_15[3].irvec, brs_15.irlabs, wyckoff) == corner_sg13(brs_13[5].irvec, brs_13[1].irlabs, wyckoff)
end

In [38]:
sg = 16
polarization = "tm"
brs = bandreps(sg, 2)
band_groupingsv16 = Vector{BandSummary}[]
for id in 1:1
    symeigsd, lgd = read_symdata("dim2-sg$sg-$id-res64-$polarization", dir="./symeigs/shiftedlattices/output");
    lgirsd = pick_lgirreps(lgd)
    fixup_gamma_symmetry!(symeigsd, lgd, Symbol(uppercase(polarization)))
    band_groupings = analyze_symmetry_data(symeigsd, lgirsd, brs);
    push!(band_groupingsv16, band_groupings)
end
brs = bandreps(2, 2)
band_groupingsv2 = Vector{BandSummary}[]
for id in 2:5
    symeigsd, lgd = read_symdata("dim2-sg$sg-$id-res64-$polarization", sgnum = 2, dir="./symeigs/shiftedlattices/output");
    lgirsd = pick_lgirreps(lgd)
    fixup_gamma_symmetry!(symeigsd, lgd, Symbol(uppercase(polarization)))
    band_groupings = analyze_symmetry_data(symeigsd, lgirsd, brs);
    push!(band_groupingsv2, band_groupings)
end

In [39]:
for grouping in band_groupingsv16[1]
    band = grouping.band
    n16 = grouping.n
    n2 = sum(band_groupingsv2[4][band]).n
    @assert n2 == [n16[1], n16[2], n16[1], n16[2], n16[1], n16[2], n16[3]+2*n16[5], n16[4]+2*n16[6], n16[9]]
end
for grouping in band_groupingsv16[1]
    band = grouping.band
    n16 = grouping.n
    n2 = sum(band_groupingsv2[1][band]).n
    @assert n2 == [n16[1], n16[2], n16[2], n16[1], n16[2], n16[1], 
        n16[3]+2*n16[5], n16[4]+2*n16[6], n16[9]]
end
for grouping in band_groupingsv16[1]
    band = grouping.band
    n16 = grouping.n
    n2 = sum(band_groupingsv2[2][band]).n
    @assert n2 == [n16[2], n16[1], n16[1], n16[2], n16[2], n16[1], 
        n16[3]+2*n16[5], n16[4]+2*n16[6], n16[9]]
end
for grouping in band_groupingsv16[1]
    band = grouping.band
    n16 = grouping.n
    n2 = sum(band_groupingsv2[3][band]).n
    @assert n2 == [n16[2], n16[1], n16[2], n16[1], n16[1], n16[2], 
        n16[3]+2*n16[5], n16[4]+2*n16[6], n16[9]]
end

In [40]:
sg = 17
polarization = "tm"
brs = bandreps(sg, 2)
band_groupingsv17 = Vector{BandSummary}[]
for id in 1:1
    symeigsd, lgd = read_symdata("dim2-sg$sg-$id-res64-$polarization", dir="./symeigs/shiftedlattices/output");
    lgirsd = pick_lgirreps(lgd)
    fixup_gamma_symmetry!(symeigsd, lgd, Symbol(uppercase(polarization)))
    band_groupings = analyze_symmetry_data(symeigsd, lgirsd, brs);
    push!(band_groupingsv17, band_groupings)
end
brs = bandreps(2, 2)
band_groupingsv2 = Vector{BandSummary}[]
for id in 2:5
    symeigsd, lgd = read_symdata("dim2-sg$sg-$id-res64-$polarization", sgnum = 2, dir="./symeigs/shiftedlattices/output");
    lgirsd = pick_lgirreps(lgd)
    fixup_gamma_symmetry!(symeigsd, lgd, Symbol(uppercase(polarization)))
    band_groupings = analyze_symmetry_data(symeigsd, lgirsd, brs, atol=1e-1);
    #println(band_groupings)
    #println(symeigsd)
    push!(band_groupingsv2, band_groupings)
end

In [41]:
for grouping in band_groupingsv17[1]
    band = grouping.band
    n17 = grouping.n
    n2 = sum(band_groupingsv2[1][band]).n
    @assert n2 == [n17[1]+n17[2], n17[3]+n17[4], n17[1]+n17[2], n17[3]+n17[4], n17[1]+n17[2], n17[3]+n17[4],
    n17[5]+n17[6]+2*n17[9], n17[7]+n17[8]+2*n17[10], n17[14]]
end
for grouping in band_groupingsv17[1]
    band = grouping.band
    n17 = grouping.n
    n2 = sum(band_groupingsv2[2][band]).n
    @assert n2 == [n17[3]+n17[4], n17[1]+n17[2], n17[1]+n17[2], n17[3]+n17[4], n17[3]+n17[4], n17[1]+n17[2],
    n17[5]+n17[6]+2*n17[9], n17[7]+n17[8]+2*n17[10], n17[14]]
end
for grouping in band_groupingsv17[1]
    band = grouping.band
    n17 = grouping.n
    n2 = sum(band_groupingsv2[3][band]).n
    @assert n2 == [n17[1]+n17[2], n17[3]+n17[4], n17[3]+n17[4], n17[1]+n17[2], n17[3]+n17[4], n17[1]+n17[2],
    n17[5]+n17[6]+2*n17[9], n17[7]+n17[8]+2*n17[10], n17[14]]
end
for grouping in band_groupingsv17[1]
    band = grouping.band
    n17 = grouping.n
    n2 = sum(band_groupingsv2[4][band]).n
    @assert n2 == [n17[3]+n17[4], n17[1]+n17[2], n17[3]+n17[4], n17[1]+n17[2], n17[1]+n17[2], n17[3]+n17[4],
    n17[5]+n17[6]+2*n17[9], n17[7]+n17[8]+2*n17[10], n17[14]]
end

In [113]:
# Below we list the decomposition of EBRS in plane group 6 and 9 as EBRs in plane group 2

In [197]:
function brs_six_to_two(br::BandRep)
    br6 = collect(br)
    n2 = [br6[1]+br6[2], br6[3]+br6[4], br6[9]+br6[10], br6[11]+br6[12], br6[5]+br6[6],
        br6[7]+br6[8], br6[13]+br6[14], br6[15]+br6[16], br6[17]]
    return decompose(n2, bandreps(2, 2))
end

function brs_nine_to_two(br::BandRep)
    br9 = collect(br)
    n2 = [br9[5], br9[6], br9[5], br9[6], br9[1]+br9[2],
        br9[3]+br9[4], br9[7]+br9[8], br9[9]+br9[10], br9[11]]
    return decompose(n2, bandreps(2, 2))
end

function brs_fourteen_to_thirteen(br::BandRep)
    br14 = collect(br)
    n13 = [br14[6], br14[8], br14[7], br14[3]+br14[4], br14[5], br14[6], br14[7], br14[8], br14[9]]
    return(decompose(n13, bandreps(13, 2)))
end

function brs_fifteen_to_thirteen(br::BandRep)
    br15 = collect(br)
    n13 = [br15[1]+br15[2], br15[3], br15[3], br15[6]+br15[7], br15[8], br15[9]+br15[10], br15[11], br15[11], br15[12]]
    return(decompose(n13, bandreps(13, 2)))
end

function brs_eleven_to_ten(br::BandRep)
    br11 = collect(br)
    n10 = [br11[1]+br11[2], br11[3]+ br11[4], br11[5]+br11[8], br11[6]+br11[7], br11[9],
        br11[10]+br11[13], br11[11]+br11[12], br11[14], br11[15]]
    return(decompose(n10, bandreps(10, 2)))
end

function brs_twelve_to_ten(br::BandRep)
    br12 = collect(br)
    n10 = [br12[4], br12[4], br12[3], br12[3], br12[1]+br12[2], br12[5]+br12[8], br12[6]+br12[7], br12[9], br12[10]]
    return(decompose(n10, bandreps(10, 2)))
end

function brs_seventeen_to_sixteen(br::BandRep)
    br17 = collect(br)
    n16 = [br17[1]+br17[2], br17[3]+br17[4], br17[5]+br17[6], br17[7]+br17[8], br17[9], br17[10], br17[11]+br17[12], 
    br17[13], br17[14]]
    return(decompose(n16, bandreps(16, 2)))
end

function brs_sixteen_to_thirteen(br::BandRep)
    br16 = collect(br)
    n16 = [br16[7],br16[8],br16[8],br16[3]+br16[4], br16[5]+br16[6], br16[7], br16[8], br16[8], br16[9]]
    return(decompose(n16, bandreps(13, 2)))
end

function brs_sixteen_to_two(br::BandRep)
    br16 = collect(br)
    n16 = [br16[1], br16[2], br16[1], br16[2], br16[1], br16[2], br16[3]+2*br16[5], br16[4]+2*br16[6]]
    return(decompose(n16, bandreps(2, 2)))
end

function brs_seventeen_to_two(br::BandRep)
   br17 = collect(br)
   n17 =  [br17[1]+br17[2], br17[3]+br17[4], br17[1]+br17[2], br17[3]+br17[4], br17[1]+br17[2], br17[3]+br17[4],
   br17[5]+br17[6]+2*br17[9], br17[7]+br17[8]+2*br17[10], br17[14]]
   return(decompose(n17, bandreps(2,2)))
end

function brs_seventeen_to_fourteen(br::BandRep)
    br17 = collect(br)
    n17 =  [br17[1]+br17[4], br17[2]+br17[3],
            br17[5]+br17[8], br17[6]+br17[7], br17[9]+br17[10],
        br17[11]+br17[12], br17[13], br17[13], br17[14]]
    return(decompose(n17, bandreps(14,2)))
end

function brs_seventeen_to_fifteen(br::BandRep)
    br17 = collect(br)
    n17 =  [br17[11], br17[12], br17[13],
        br17[1]+br17[3], br17[2]+br17[4],
        br17[5]+br17[7], br17[6]+br17[8], br17[9]+br17[10],
        br17[11], br17[12], br17[13], br17[14]]
    return(decompose(n17, bandreps(15,2)))
end

brs_seventeen_to_fifteen (generic function with 1 method)

In [69]:
fifteen_ebrs = brs_seventeen_to_fifteen.(bandreps(17, 2))
brs = bandreps(15, 2)
brs_labs = ["(" * br.wyckpos * "|"  * prod(string.(collect(br.label)[1:end-2])) * ")" for br in brs]
Crystalline.symvec2string.(fifteen_ebrs, Ref(brs_labs))

13-element Vector{String}:
 "[(1a|A₁)+(1a|E)]"
 "[(1a|A₂)+(1a|E)]"
 "[(1a|A₁)+(1a|E)]"
 "[(1a|A₂)+(1a|E)]"
 "[(2b|A)]"
 "[(2b|A)]"
 "[(2b|E)]"
 "[(1a|A₁)]"
 "[(1a|A₂)]"
 "[(1a|A₁)]"
 "[(1a|A₂)]"
 "[(1a|E)]"
 "[(1a|E)]"

In [44]:
fourteen_ebrs = brs_seventeen_to_fourteen.(bandreps(17, 2))
brs = bandreps(14, 2)
brs_labs = ["(" * br.wyckpos * "|"  * prod(string.(collect(br.label)[1:end-2])) * ")" for br in brs]
Crystalline.symvec2string.(fourteen_ebrs, Ref(brs_labs))

13-element Vector{String}:
 "[(1c|A₁)+(1c|E)]"
 "[(1c|A₂)+(1c|E)]"
 "[(1c|A₂)+(1c|E)]"
 "[(1c|A₁)+(1c|E)]"
 "[(1c|A₁)+(1b|A₁)]"
 "[(1c|A₂)+(1b|A₂)]"
 "[(1c|E)+(1b|E)]"
 "[(1a|A₁)]"
 "[(1a|A₂)]"
 "[(1a|A₂)]"
 "[(1a|A₁)]"
 "[(1a|E)]"
 "[(1a|E)]"

In [111]:
#Check that corner charges and polarizations make sense
for br6 in bandreps(6, 2)
     @assert corner_sg6(br6.irvec, br6.irlabs) == corner_sg2(hcat(bandreps(2, 2).bandreps...)*Int.(brs_six_to_two(br6)),
            bandreps(2, 2).irlabs)
    @assert polarization_sg6(br6.irvec, br6.irlabs) == polarization_sg2(hcat(bandreps(2, 2).bandreps...)*Int.(brs_six_to_two(br6)),
            bandreps(2, 2).irlabs)
end

In [119]:
for br9 in bandreps(9, 2)
    @assert corner_sg9(br9.irvec, br9.irlabs) == corner_sg2(hcat(bandreps(2, 2).bandreps...)*Int.(brs_nine_to_two(br9)),
            bandreps(2, 2).irlabs)
    @assert polarization_sg9(br9.irvec, br9.irlabs) == polarization_sg2(hcat(bandreps(2, 2).bandreps...)*Int.(brs_nine_to_two(br9)),
            bandreps(2, 2).irlabs)
end

In [153]:
for br14 in bandreps(14, 2)
     @assert corner_sg14(br14.irvec, br14.irlabs) == corner_sg13(hcat(bandreps(13, 2).bandreps...)*Int.(brs_fourteen_to_thirteen(br14)),
            bandreps(13, 2).irlabs)
     @assert polarization_sg14(br14.irvec, br14.irlabs) == polarization_sg13(hcat(bandreps(13, 2).bandreps...)*Int.(brs_fourteen_to_thirteen(br14)),
            bandreps(13, 2).irlabs)
end

In [184]:
for br11 in bandreps(11, 2)
     @assert corner_sg11(br11.irvec, br11.irlabs) == corner_sg10(hcat(bandreps(10, 2).bandreps...)*Int.(brs_eleven_to_ten(br11)),
            bandreps(10, 2).irlabs)
     @assert polarization_sg11(br11.irvec, br11.irlabs) == polarization_sg10(hcat(bandreps(10, 2).bandreps...)*Int.(brs_eleven_to_ten(br11)),
            bandreps(10, 2).irlabs)
end

In [199]:
for br12 in bandreps(12, 2)
     @assert corner_sg12(br12.irvec, br12.irlabs) == corner_sg10(hcat(bandreps(10, 2).bandreps...)*Int.(brs_twelve_to_ten(br12)),
            bandreps(10, 2).irlabs)
         @assert polarization_sg12(br12.irvec, br12.irlabs) == polarization_sg10(hcat(bandreps(10, 2).bandreps...)*Int.(brs_twelve_to_ten(br12)),
            bandreps(10, 2).irlabs)
end

In [221]:
for br17 in bandreps(17, 2)
     @assert corner_sg17(br17.irvec, br17.irlabs) == corner_sg16(hcat(bandreps(16, 2).bandreps...)*Int.(brs_seventeen_to_sixteen(br17)),
            bandreps(16, 2).irlabs)
     @assert polarization_sg17(br17.irvec, br17.irlabs) == polarization_sg16(hcat(bandreps(16, 2).bandreps...)*Int.(brs_seventeen_to_sixteen(br17)),
            bandreps(16, 2).irlabs)
end

In [24]:
# Below we see how EBRs change when the wyckoff position is changed

# We do not consider sg16, sg17, and sg12 since in the first two cases, we can't recenter to a 6-fold symmetric
# Wyckoff position and in the last case the EBRs do not change due to the nonsymmorphic symmetry. 

In [195]:
function brs_move_sg2(br::BandRep, wp::WyckoffPosition{2})
    n = collect(br)
    return brs_move_sg2(n, wp)
end

function brs_move_sg6(br::BandRep, wp::WyckoffPosition{2})
    n = collect(br)
    return brs_move_sg6(n, wp)
end

function brs_move_sg9(br::BandRep, wp::WyckoffPosition{2})
    n = collect(br)
    return brs_move_sg9(n, wp)
end

function brs_move_sg10(br::BandRep, wp::WyckoffPosition{2})
    n = collect(br)
    return brs_move_sg10(n, wp)
end

function brs_move_sg11(br::BandRep, wp::WyckoffPosition{2})
    n = collect(br)
    return brs_move_sg11(n, wp)
end

function brs_move_sg13(br::BandRep, wp::WyckoffPosition{2}, flip_k::Bool=true)
    n = collect(br)
    return brs_move_sg13(n, wp, flip_k)
end

function brs_move_sg14(br::BandRep, wp::WyckoffPosition{2}, flip_k::Bool=true)
    n = collect(br)
    return brs_move_sg14(n, wp, flip_k)
end

function brs_move_sg2(n::Vector{<:Integer}, wp::WyckoffPosition{2})
    brs = bandreps(2, 2)
    wps = wyckoffs(2, 2)
    shift = if wp == wps[end-1]
                n[[2, 1, 3, 4, 6, 5, 7, 8, 9]]
            elseif wp == wps[end-2]
                n[[1, 2, 4, 3, 6, 5, 7, 8, 9]]
            elseif wp == wps[end-3]
                n[[2, 1, 4, 3, 5, 6, 7, 8, 9]]
            end
    decompose(shift, brs)
end

function brs_move_sg6(n::Vector{<:Integer}, wp::WyckoffPosition{2})
    brs = bandreps(6, 2)
    wps = wyckoffs(6, 2)
    shift = if wp == wps[end-1]
                n[[4, 3, 2, 1, 8, 7, 6, 5, 9, 10, 11, 12, 13, 14, 15, 16, 17]]
            elseif wp == wps[end-2]
                n[[1, 2, 3, 4, 7, 8, 5, 6, 11, 12, 9, 10, 13, 14, 15, 16, 17]]
            elseif wp == wps[end-3]
                n[[4, 3, 2, 1, 6, 5, 8, 7, 11, 12, 9, 10, 13, 14, 15, 16, 17]]
            end
    decompose(shift, brs)
end

function brs_move_sg9(n::Vector{<:Integer})
    brs = bandreps(9, 2)
    wps = wyckoffs(9, 2)
    shift = n[[1, 2, 3, 4, 6, 5, 7, 8, 9, 10, 11]]
    decompose(shift, brs)
end

function brs_move_sg10(n::Vector{<:Integer})
    brs = bandreps(10, 2)
    shift = n[[2, 1, 4, 3, 5, 6, 7, 8, 9]]
    return decompose(shift, brs)
end

function brs_move_sg11(n::Vector{<:Integer})
    brs = bandreps(11, 2)
    shift = n[[4, 3, 2, 1, 7, 8, 5, 6, 9, 10, 11, 12, 13, 14, 15]]
    return decompose(shift, brs)
end

function brs_move_sg13(n::Vector{<:Integer}, wp::WyckoffPosition{2}, flip_k::Bool=true)
    brs = bandreps(13, 2)
    wps = wyckoffs(13, 2)

    n = !flip_k ? n : n[[6, 7, 8, 4, 5, 1, 2, 3, 9]] # Swith K <-> KA irreps
    # The idea here is that the true band reps correspond to this switch, due to our convention for the symmetry eigenvalues
    
    shift = if wp == wps[end-1] 
                n[[2, 3, 1, 4, 5, 8, 6, 7, 9]]
            elseif wp == wps[end-2]
                n[[3, 1, 2, 4, 5, 7, 8, 6, 9]]
            end
    shift = !flip_k ? shift : shift[[6, 7, 8, 4, 5, 1, 2, 3, 9]] # Swith K <-> KA irreps

    return decompose(shift, brs)
end

function brs_move_sg14(n::Vector{<:Integer}, wp::WyckoffPosition{2}, flip_k::Bool=true)
    brs = bandreps(14, 2)
    wps = wyckoffs(14, 2)
    n = !flip_k ? n : n[[1, 2, 3, 4, 5, 6, 8, 7, 9]] # Swith K <-> KA irreps

    shift = if wp == wps[end-1] 
                n[[1, 2, 3, 4, 5, 8, 6, 7, 9]]
            elseif wp == wps[end-2]
                n[[1, 2, 3, 4, 5, 7, 8, 6, 9]]
            end
    
    shift = !flip_k ? shift : shift[[1, 2, 3, 4, 5, 6, 8, 7, 9]] # Swith K <-> KA irreps
    return decompose(shift, brs)
end

function brs_move_sg15(br::BandRep, wp::WyckoffPosition{2}, flip_k::Bool = true)
    sg13_decomposition = brs_fifteen_to_thirteen(br)
    n13 = hcat(bandreps(13, 2)...)*sg13_decomposition
    return brs_move_sg13(Int.(n13), wp, flip_k)
end

brs_move_sg15 (generic function with 4 methods)

In [155]:
sgnum = 2
brs = bandreps(sgnum, 2)
wps = wyckoffs(sgnum, 2)

for wp in wps[2:4]
    println("Recentering to wyckoff position: ", wp)
    for br in brs
        new_ebrs_decomposition = brs_move_sg2(br, wp)
        new_ebr = brs[findfirst(x->!iszero(x), new_ebrs_decomposition)]
        println("Bandrep at wyckoff position $(br.wyckpos) is recentered to $(new_ebr.wyckpos)")
    end
    println("")
end

Recentering to wyckoff position: 1d: [1/2, 1/2]
Bandrep at wyckoff position 1d is recentered to 1a
Bandrep at wyckoff position 1d is recentered to 1a
Bandrep at wyckoff position 1c is recentered to 1b
Bandrep at wyckoff position 1c is recentered to 1b
Bandrep at wyckoff position 1b is recentered to 1c
Bandrep at wyckoff position 1b is recentered to 1c
Bandrep at wyckoff position 1a is recentered to 1d
Bandrep at wyckoff position 1a is recentered to 1d

Recentering to wyckoff position: 1c: [1/2, 0]
Bandrep at wyckoff position 1d is recentered to 1b
Bandrep at wyckoff position 1d is recentered to 1b
Bandrep at wyckoff position 1c is recentered to 1a
Bandrep at wyckoff position 1c is recentered to 1a
Bandrep at wyckoff position 1b is recentered to 1d
Bandrep at wyckoff position 1b is recentered to 1d
Bandrep at wyckoff position 1a is recentered to 1c
Bandrep at wyckoff position 1a is recentered to 1c

Recentering to wyckoff position: 1b: [0, 1/2]
Bandrep at wyckoff position 1d is recenter

In [159]:
sgnum = 6
brs = bandreps(sgnum, 2)
wps = wyckoffs(sgnum, 2)

for wp in wps[end-3:end-1]
    println("Recentering to wyckoff position: ", wp)
    for br in brs
        new_ebrs_decomposition = brs_move_sg6(br, wp)
        new_ebr = brs[findfirst(x->!iszero(x), new_ebrs_decomposition)]
        println("Bandrep at wyckoff position $(br.wyckpos) is recentered to $(new_ebr.wyckpos)")
    end
    println("")
end

Recentering to wyckoff position: 1d: [1/2, 1/2]
Bandrep at wyckoff position 1d is recentered to 1a
Bandrep at wyckoff position 1d is recentered to 1a
Bandrep at wyckoff position 1d is recentered to 1a
Bandrep at wyckoff position 1d is recentered to 1a
Bandrep at wyckoff position 1c is recentered to 1b
Bandrep at wyckoff position 1c is recentered to 1b
Bandrep at wyckoff position 1c is recentered to 1b
Bandrep at wyckoff position 1c is recentered to 1b
Bandrep at wyckoff position 1b is recentered to 1c
Bandrep at wyckoff position 1b is recentered to 1c
Bandrep at wyckoff position 1b is recentered to 1c
Bandrep at wyckoff position 1b is recentered to 1c
Bandrep at wyckoff position 1a is recentered to 1d
Bandrep at wyckoff position 1a is recentered to 1d
Bandrep at wyckoff position 1a is recentered to 1d
Bandrep at wyckoff position 1a is recentered to 1d

Recentering to wyckoff position: 1c: [1/2, 0]
Bandrep at wyckoff position 1d is recentered to 1b
Bandrep at wyckoff position 1d is rece

In [162]:
sgnum = 10
brs = bandreps(sgnum, 2)
wps = wyckoffs(sgnum, 2)

for wp in wps[end-1:end-1]
    println("Recentering to wyckoff position: ", wp)
    for br in brs
        new_ebrs_decomposition = brs_move_sg10(br)
        new_ebr = brs[findfirst(x->!iszero(x), new_ebrs_decomposition)]
        println("Bandrep at wyckoff position $(br.wyckpos) is recentered to $(new_ebr.wyckpos)")
    end
    println("")
end

Recentering to wyckoff position: 1b: [1/2, 1/2]
Bandrep at wyckoff position 2c is recentered to 2c
Bandrep at wyckoff position 2c is recentered to 2c
Bandrep at wyckoff position 1b is recentered to 1a
Bandrep at wyckoff position 1b is recentered to 1a
Bandrep at wyckoff position 1b is recentered to 1a
Bandrep at wyckoff position 1a is recentered to 1b
Bandrep at wyckoff position 1a is recentered to 1b
Bandrep at wyckoff position 1a is recentered to 1b



In [164]:
sgnum = 11
brs = bandreps(sgnum, 2)
wps = wyckoffs(sgnum, 2)

for wp in wps[end-1:end-1]
    println("Recentering to wyckoff position: ", wp)
    for br in brs
        new_ebrs_decomposition = brs_move_sg11(br)
        new_ebr = brs[findfirst(x->!iszero(x), new_ebrs_decomposition)]
        println("Bandrep at wyckoff position $(br.wyckpos) is recentered to $(new_ebr.wyckpos)")
    end
    println("")
end

Recentering to wyckoff position: 1b: [1/2, 1/2]
Bandrep at wyckoff position 2c is recentered to 2c
Bandrep at wyckoff position 2c is recentered to 2c
Bandrep at wyckoff position 2c is recentered to 2c
Bandrep at wyckoff position 2c is recentered to 2c
Bandrep at wyckoff position 1b is recentered to 1a
Bandrep at wyckoff position 1b is recentered to 1a
Bandrep at wyckoff position 1b is recentered to 1a
Bandrep at wyckoff position 1b is recentered to 1a
Bandrep at wyckoff position 1b is recentered to 1a
Bandrep at wyckoff position 1a is recentered to 1b
Bandrep at wyckoff position 1a is recentered to 1b
Bandrep at wyckoff position 1a is recentered to 1b
Bandrep at wyckoff position 1a is recentered to 1b
Bandrep at wyckoff position 1a is recentered to 1b



In [182]:
sgnum = 13
brs = bandreps(sgnum, 2)
wps = wyckoffs(sgnum, 2)

for flip_k in [true, false]
    println("flip_k: $(flip_k)\n")
    for wp in wps[end-2:end-1]
        println("Recentering to wyckoff position: ", wp)
        for br in brs
            new_ebrs_decomposition = brs_move_sg13(br, wp, flip_k)
            new_ebr = brs[findfirst(x->!iszero(x), new_ebrs_decomposition)]
            println("Bandrep at wyckoff position $(br.wyckpos) is recentered to $(new_ebr.wyckpos)")
        end
        println("")
    end
end

flip_k: true

Recentering to wyckoff position: 1c: [2/3, 1/3]
Bandrep at wyckoff position 1c is recentered to 1a
Bandrep at wyckoff position 1c is recentered to 1a
Bandrep at wyckoff position 1b is recentered to 1c
Bandrep at wyckoff position 1b is recentered to 1c
Bandrep at wyckoff position 1a is recentered to 1b
Bandrep at wyckoff position 1a is recentered to 1b

Recentering to wyckoff position: 1b: [1/3, 2/3]
Bandrep at wyckoff position 1c is recentered to 1b
Bandrep at wyckoff position 1c is recentered to 1b
Bandrep at wyckoff position 1b is recentered to 1a
Bandrep at wyckoff position 1b is recentered to 1a
Bandrep at wyckoff position 1a is recentered to 1c
Bandrep at wyckoff position 1a is recentered to 1c

flip_k: false

Recentering to wyckoff position: 1c: [2/3, 1/3]
Bandrep at wyckoff position 1c is recentered to 1b
Bandrep at wyckoff position 1c is recentered to 1b
Bandrep at wyckoff position 1b is recentered to 1a
Bandrep at wyckoff position 1b is recentered to 1a
Bandrep a

In [172]:
wyckoffs(14, 2)

5-element Vector{WyckoffPosition{2}}:
 6e: [α, β]
 3d: [α, -α]
 1c: [2/3, 1/3]
 1b: [1/3, 2/3]
 1a: [0, 0]

In [204]:
sgnum = 14
brs = bandreps(sgnum, 2)
wps = wyckoffs(sgnum, 2)

for flip_k in [true, false]
    for wp in wps[end-2:end-1]
        println("Recentering to wyckoff position: ", wp)
        for br in brs
            new_ebrs_decomposition = brs_move_sg14(br, wp, flip_k)
            new_ebr = brs[findfirst(x->!iszero(x), new_ebrs_decomposition)]
            println("Bandrep at wyckoff position $(br.wyckpos) is recentered to $(new_ebr.wyckpos)")
        end
        println("")
    end
end

Recentering to wyckoff position: 1c: [2/3, 1/3]
Bandrep at wyckoff position 1c is recentered to 1a
Bandrep at wyckoff position 1c is recentered to 1a
Bandrep at wyckoff position 1c is recentered to 1a
Bandrep at wyckoff position 1b is recentered to 1c
Bandrep at wyckoff position 1b is recentered to 1c
Bandrep at wyckoff position 1b is recentered to 1c
Bandrep at wyckoff position 1a is recentered to 1b
Bandrep at wyckoff position 1a is recentered to 1b
Bandrep at wyckoff position 1a is recentered to 1b

Recentering to wyckoff position: 1b: [1/3, 2/3]
Bandrep at wyckoff position 1c is recentered to 1b
Bandrep at wyckoff position 1c is recentered to 1b
Bandrep at wyckoff position 1c is recentered to 1b
Bandrep at wyckoff position 1b is recentered to 1a
Bandrep at wyckoff position 1b is recentered to 1a
Bandrep at wyckoff position 1b is recentered to 1a
Bandrep at wyckoff position 1a is recentered to 1c
Bandrep at wyckoff position 1a is recentered to 1c
Bandrep at wyckoff position 1a is re

In [203]:
sgnum = 15
brs = bandreps(sgnum, 2)
brs13 = bandreps(13, 2)
wps = wyckoffs(13, 2)

for flip_k in [true, false]
    for wp in wps[end-2:end-1]
        println("Recentering to wyckoff position: ", wp)
        for br in brs
            new_ebrs_decomposition = brs_move_sg15(br, wp, flip_k)
            new_ebrs = brs13[findall(x->!iszero(x), new_ebrs_decomposition)]
            println("Bandrep at wyckoff position $(br.wyckpos) is recentered to $([new_ebr.wyckpos for new_ebr in new_ebrs])")
        end
        println("")
    end
end

Recentering to wyckoff position: 1c: [2/3, 1/3]
Bandrep at wyckoff position 2b is recentered to ["1c", "1a"]
Bandrep at wyckoff position 2b is recentered to ["1c", "1a"]
Bandrep at wyckoff position 1a is recentered to ["1b"]
Bandrep at wyckoff position 1a is recentered to ["1b"]
Bandrep at wyckoff position 1a is recentered to ["1b"]

Recentering to wyckoff position: 1b: [1/3, 2/3]
Bandrep at wyckoff position 2b is recentered to ["1b", "1a"]
Bandrep at wyckoff position 2b is recentered to ["1b", "1a"]
Bandrep at wyckoff position 1a is recentered to ["1c"]
Bandrep at wyckoff position 1a is recentered to ["1c"]
Bandrep at wyckoff position 1a is recentered to ["1c"]

Recentering to wyckoff position: 1c: [2/3, 1/3]
Bandrep at wyckoff position 2b is recentered to ["1b", "1a"]
Bandrep at wyckoff position 2b is recentered to ["1b", "1a"]
Bandrep at wyckoff position 1a is recentered to ["1c"]
Bandrep at wyckoff position 1a is recentered to ["1c"]
Bandrep at wyckoff position 1a is recentered to 